In [2]:
from datetime import datetime, date
from pydantic import BaseModel, constr, field_validator

def parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%d/%m/%y").date()
    except ValueError as e:
        raise ValueError("Incorrect date format, should be DD/MM/YY") from e

class MyModel(BaseModel):
    date_field: date

    @field_validator("date_field")
    def parse_date_field(cls, v):
        return parse_date(v)

# Example usage:
data = {"date_field": "07/05/24"}
model_instance = MyModel(**data)
print(model_instance.model_dump())


date_field=datetime.date(2024, 5, 7)
